In [ ]:
!pip install langchain PyPDF2 sentence-transformers faiss-cpu langchain_community torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
WORKING_PATH = '/content/drive/MyDrive/Colab Notebooks/AgentesIA'
%cd {WORKING_PATH}

import sys
sys.path.append(WORKING_PATH)

/content/drive/MyDrive/Colab Notebooks/AgentesIA


In [ ]:
data_directory = "/content/drive/MyDrive/Colab Notebooks/AgentesIA/data"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from tqdm import tqdm
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
if torch.cuda.is_available():
    print("¡GPU detectada!:", torch.cuda.get_device_name(0))
else:
    print("No se encontró GPU. Se usará CPU.")

No se encontró GPU. Se usará CPU.


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
).to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct")

configuration_phi3.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/54.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def read_pdf(file_path):
    """Lee el contenido de un archivo PDF y devuelve el texto extraído."""
    pdf_reader = PdfReader(file_path)
    text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
    return text

pdf_files = [f for f in os.listdir(data_directory) if f.endswith(".pdf")]

documents = []
for pdf_file in tqdm(pdf_files, desc="Procesando PDFs"):
    pdf_text = read_pdf(os.path.join(data_directory, pdf_file))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_text(pdf_text)
    documents.extend(chunks)

if documents:
    vectorstore = FAISS.from_texts(documents, embedder)
else:
    raise ValueError("No se encontraron documentos para indexar.")

def find_most_relevant_document(question):
    """Encuentra los fragmentos de texto más relevantes para la pregunta."""
    docs = vectorstore.similarity_search(question, k=3)
    return "\n".join([doc.page_content for doc in docs])

Procesando PDFs: 100%|██████████| 3/3 [00:02<00:00,  1.43it/s]


In [ ]:
def get_answer(question, db, model, tokenizer):
    docs = db.similarity_search(question)
    context = " ".join([doc.page_content for doc in docs])

    prompt = f"Contexto: {context}\nPregunta: {question}\nRespuesta:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=256)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer.split("Respuesta:")[1].strip()

In [ ]:
while True:
    pregunta = input("Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): ")
    if pregunta.lower() in ['salir', 'exit', 'quit']:
        break

    contexto = find_most_relevant_document(pregunta)

    prompt = f"Contexto: {contexto}\nPregunta: {question}\nRespuesta:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=256)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(answer.split("Respuesta:")[1].strip())